In [1]:
region='ap-northeast-2'
bucket = 'daytrip-ai-service'
role = "arn:aws:iam::742627718059:role/DayTripAISagemaker"

print('Region: {}'.format(region))
print('Bucket: {}'.format(bucket))
print('Role: {}'.format(role))

Region: ap-northeast-2
Bucket: daytrip-ai-service
Role: arn:aws:iam::742627718059:role/DayTripAISagemaker


In [2]:
dataset = "daylog-pref-v3"
mlp_layer = 4
predictive_factor = 64
model_name = f'neumf-{dataset}-{mlp_layer}-{predictive_factor}'
pascal_model_name = 'NeuMFDaylogPrefV3'

model_package_group_name = model_name
project_prefix = model_name

print('Model package group name: {}'.format(model_package_group_name))
print('Project prefix: {}'.format(project_prefix))

Model package group name: neumf-daylog-pref-v3-4-64
Project prefix: neumf-daylog-pref-v3-4-64


In [3]:
import boto3
from sagemaker.workflow.pipeline_context import PipelineSession

boto_session = boto3.Session(region_name=region)
pipeline_session = PipelineSession(boto_session=boto_session, default_bucket=bucket)

sagemaker.config INFO - Not applying SDK defaults from location: /opt/homebrew/share/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/sookhyun/Library/Application Support/sagemaker/config.yaml


In [4]:
approve_model_code = pipeline_session.upload_data('./approve_model.py', bucket, f'{project_prefix}/source')
deploy_model_code = pipeline_session.upload_data('./deploy_model.py', bucket, f'{project_prefix}/source')
print('Approve Model Code: {}'.format(approve_model_code))
print('Deploy Model Code: {}'.format(deploy_model_code))

Approve Model Code: s3://daytrip-ai-service/neumf-daylog-pref-v3-4-64/source/approve_model.py
Deploy Model Code: s3://daytrip-ai-service/neumf-daylog-pref-v3-4-64/source/deploy_model.py


In [5]:
from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutputTypeEnum, LambdaOutput
from sagemaker.lambda_helper import Lambda

func_approve_model = Lambda(
    function_name='sagemaker-approve-model',
    execution_role_arn=role,
    script='approve_model.py',
    handler='approve_model.handler',
)

model_approve_step = LambdaStep(
    name=pascal_model_name+'-Approve',
    lambda_func=func_approve_model,
    inputs={
        'model_package_group_name': model_package_group_name,
    },
    outputs=[
        LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String),
        LambdaOutput(output_name="modelPackageArn", output_type=LambdaOutputTypeEnum.String),
    ],
)

In [6]:
from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutputTypeEnum, LambdaOutput
from sagemaker.lambda_helper import Lambda

func_deploy_model = Lambda(
    function_name='sagemaker-deploy-model',
    execution_role_arn=role,
    script='deploy_model.py',
    handler='deploy_model.handler',
)

model_deploy_step = LambdaStep(
    name=pascal_model_name+'-Deploy',
    lambda_func=func_deploy_model,
    inputs={
        'model_name': model_name,
        'role': role,
        'model_package_arn': model_approve_step.properties.Outputs['modelPackageArn'],
        'instance_type': 'ml.g4dn.4xlarge',
    },
    outputs=[],
)

In [7]:
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import ConditionEquals

condition_step = ConditionStep(
    name=pascal_model_name+'-ApproveStatusCheck',
    conditions=[
        ConditionEquals(
            left=model_approve_step.properties.Outputs['statusCode'],
            right=200,
        ),
    ],
    if_steps=[model_deploy_step],
    else_steps=[],
    depends_on=[model_approve_step],
)


In [8]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = pascal_model_name + '-Deploy-Pipeline'
pipeline = Pipeline(
    name=pipeline_name,
    sagemaker_session=pipeline_session,
    steps=[model_approve_step, condition_step],
)

pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:742627718059:pipeline/NeuMFDaylogPrefV3-Deploy-Pipeline',
 'ResponseMetadata': {'RequestId': '68ad45e9-2be9-4903-ad7b-205c0285a12c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '68ad45e9-2be9-4903-ad7b-205c0285a12c',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '106',
   'date': 'Thu, 22 Aug 2024 06:57:40 GMT'},
  'RetryAttempts': 0}}

In [9]:
pipeline.start()

_PipelineExecution(arn='arn:aws:sagemaker:ap-northeast-2:742627718059:pipeline/NeuMFDaylogPrefV3-Deploy-Pipeline/execution/fx1t47xibeaw', sagemaker_session=<sagemaker.workflow.pipeline_context.PipelineSession object at 0x107f4bf70>)